<a href="https://colab.research.google.com/github/Kaveriharamagatti/NVIDIA_async_detection/blob/main/Methodology_IV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#CROPPING - EXTRACTING REGION OF INTEREST
import os
import cv2

# Read the image
image = cv2.imread('/content/drive/MyDrive/cropping_model/2.png')

# Get image dimensions
height, width, _ = image.shape

# Define the dimensions of the black box containing LEDs (assuming it's centered)
# Assuming each LED has a diameter of 10 pixels
led_diameter = 10
led_spacing = 1  # Spacing between LEDs
num_leds = 11  # Number of LEDs in each row and column

black_box_width = num_leds * (led_diameter + led_spacing) - led_spacing
black_box_height = num_leds * (led_diameter + led_spacing) - led_spacing

# Calculate the original bounding box coordinates
x1 = (width - black_box_width) // 2
y1 = (height - black_box_height) // 2
x2 = x1 + black_box_width
y2 = y1 + black_box_height

# Define individual margins for each side
margin_left = 12
margin_right = 12
margin_up = 18
margin_down = 12

# Adjust the bounding box coordinates with the specified margins
x1 = max(x1 - margin_left, 0)
y1 = max(y1 - margin_up, 0)
x2 = min(x2 + margin_right, width)
y2 = min(y2 + margin_down, height)

# Draw the increased bounding box on the image
cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Specify the output folder path
output_folder = '/content/drive/MyDrive/cropped_output'

# Create the output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Specify the output path including the filename and extension
output_path = os.path.join(output_folder, 'output_image.png')

# Store the image with the bounding box
cv2.imwrite(output_path, image)


In [ ]:
#RETAINING REGION OF INTEREST
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
image = cv2.imread('/content/drive/MyDrive/cropped_output/output_image.png')

# Convert the image to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define lower and upper bounds for the green color
lower_green = np.array([40, 40, 40])  # Lower bound for green color in HSV
upper_green = np.array([70, 255, 255])  # Upper bound for green color in HSV

# Threshold the HSV image to get only green colors
mask = cv2.inRange(hsv, lower_green, upper_green)

# Find contours
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a mask for the largest contour (assuming it's the green box)
largest_contour = max(contours, key=cv2.contourArea)
mask = np.zeros_like(mask)
cv2.drawContours(mask, [largest_contour], -1, (255), thickness=cv2.FILLED)

# Bitwise AND mask and original image
result = cv2.bitwise_and(image, image, mask=mask)

# Show the result
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Original Image')
plt.axis('off')
plt.show()

plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
plt.title('Result with Mask Applied')
plt.axis('off')
plt.show()


In [ ]:
#CROPPING REGION OF INTEREST
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
image = cv2.imread('/content/drive/MyDrive/cropped_output/output_image.png')

# Convert the image to HSV color space
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define lower and upper bounds for the green color
lower_green = np.array([40, 40, 40])  # Lower bound for green color in HSV
upper_green = np.array([70, 255, 255])  # Upper bound for green color in HSV

# Threshold the HSV image to get only green colors
mask = cv2.inRange(hsv, lower_green, upper_green)

# Find contours
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Get the bounding box of the largest contour (assuming it's the green box)
x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))

# Crop the region inside the bounding box
cropped_region = image[y:y+h, x:x+w]

# Save the cropped region as an output image
output_path = '/content/drive/MyDrive/cropped_output/cropped_region.png'
cv2.imwrite(output_path, cropped_region)

print("Cropped region saved successfully.")

# Show the cropped region
plt.imshow(cv2.cvtColor(cropped_region, cv2.COLOR_BGR2RGB))
plt.title('Cropped Region')
plt.axis('off')
plt.show()


In [ ]:
#TEMPLATE MATCHING AND CROPPING MATCHED REGION
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the main image
main_image = cv2.imread('/content/drive/MyDrive/cropped_output/cropped_region.png')

# Load the template image
template = cv2.imread('/content/drive/MyDrive/cropped_data/4.png')

# Check if the images are loaded successfully
if main_image is None or template is None:
    print("Error: Could not load the images.")
    exit()

# Convert both images to grayscale
main_gray = cv2.cvtColor(main_image, cv2.COLOR_BGR2GRAY)
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

# Perform template matching
result = cv2.matchTemplate(main_gray, template_gray, cv2.TM_CCOEFF_NORMED)

# Define a threshold to consider a match
threshold = 0.8

# Find locations where the result is above the threshold
locations = np.where(result >= threshold)

# If there are no matches, print "False" and exit
if len(locations[0]) == 0:
    print("False")
    exit()

# Loop over all the locations and mark the matches
for loc in zip(*locations[::-1]):
    cv2.rectangle(main_image, loc, (loc[0] + template.shape[1], loc[1] + template.shape[0]), (0, 255, 0), 2)

# Show the result using matplotlib
plt.imshow(cv2.cvtColor(main_image, cv2.COLOR_BGR2RGB))
plt.title('Template Matching Result')
plt.axis('off')
plt.show()

# Extract the first matched region
top_left = (locations[1][0], locations[0][0])
bottom_right = (top_left[0] + template.shape[1], top_left[1] + template.shape[0])
matched_region = main_image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]

# Resize the matched region to the specified dimensions (706x463)
output_width, output_height = 706, 463
resized_matched_region = cv2.resize(matched_region, (output_width, output_height))

# Save the resized matched region to a file
cv2.imwrite('/content/drive/MyDrive/cropped_output/matched_region.png', resized_matched_region)

# Print "True"
print("True")

# Display the resized cropped image
plt.imshow(cv2.cvtColor(resized_matched_region, cv2.COLOR_BGR2RGB))
plt.title('Matched Region')
plt.axis('off')
plt.show()


In [ ]:
#RESIZING
import os
from google.colab.patches import cv2_imshow
import cv2

# Function to resize images in a directory
def resize_images(input_folder, output_folder, desired_width, desired_height):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List all files in the input folder
    image_files = os.listdir(input_folder)

    # Resize each image and save it in the output folder
    for file_name in image_files:
        # Read the image
        image_path = os.path.join(input_folder, file_name)
        image = cv2.imread(image_path)

        # Resize the image
        resized_image = cv2.resize(image, (desired_width, desired_height))

        # Save the resized image to the output folder
        output_path = os.path.join(output_folder, file_name)
        cv2.imwrite(output_path, resized_image)

        # Display the resized image
        cv2_imshow(resized_image)

# Define input and output folders
input_folder = '/content/drive/MyDrive/Amit_sir2'
output_folder = '/content/drive/MyDrive/cropped_ouput'

# Define the desired width and height
desired_width = 200
desired_height = 180

# Resize images in the input folder and save them in the output folder
resize_images(input_folder, output_folder, desired_width, desired_height)


In [ ]:
#RETAINING WHITE AND RED REGIONS
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

# Function to process each image
def process_image(image_path, output_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to the HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the range of bright red color in HSV
    lower_bright_red1 = np.array([0, 150, 150])
    upper_bright_red1 = np.array([10, 255, 255])
    lower_bright_red2 = np.array([170, 150, 150])
    upper_bright_red2 = np.array([180, 255, 255])

    # Define the range of white color in HSV
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 25, 255])

    # Create masks for the bright red and white LEDs
    mask_bright_red1 = cv2.inRange(hsv_image, lower_bright_red1, upper_bright_red1)
    mask_bright_red2 = cv2.inRange(hsv_image, lower_bright_red2, upper_bright_red2)
    mask_bright_red = cv2.bitwise_or(mask_bright_red1, mask_bright_red2)
    mask_white = cv2.inRange(hsv_image, lower_white, upper_white)
    mask_leds = cv2.bitwise_or(mask_bright_red, mask_white)

    # Invert the mask to get the background
    mask_background = cv2.bitwise_not(mask_leds)

    # Darken the background by reducing its brightness
    background = cv2.bitwise_and(image, image, mask=mask_background)
    background = cv2.addWeighted(background, 0.5, np.zeros_like(background), 0.5, 0)

    # Combine the darkened background with the original LEDs
    result_image = cv2.bitwise_or(background, cv2.bitwise_and(image, image, mask=mask_leds))

    # Save the result
    cv2.imwrite(output_path, result_image)

    # Return the result for display
    return result_image

# Paths to input images
image_paths = [
    '/content/drive/MyDrive/cropped_ouput/1.png',
    '/content/drive/MyDrive/cropped_ouput/2.png'
]

# Output directory
output_directory = '/content/drive/MyDrive/pre_preocessed'
os.makedirs(output_directory, exist_ok=True)

# Process and display each image
results = []
for i, image_path in enumerate(image_paths):
    output_path = os.path.join(output_directory, f'Image{i+1}_Bright_Red_and_White_LEDs_Enhanced.jpg')
    result = process_image(image_path, output_path)
    results.append(result)

# Display the results
plt.figure(figsize=(10, 5))
for i, result_image in enumerate(results):
    plt.subplot(1, 2, i+1)
    plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    plt.title(f'Image {i+1}: Bright Red and White LEDs Enhanced')
    plt.axis('off')
plt.show()


In [ ]:
#REMOVING BACKGROUND
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os

def segment_image(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to the HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the range of bright red color in HSV
    lower_bright_red1 = np.array([0, 150, 150])
    upper_bright_red1 = np.array([10, 255, 255])
    lower_bright_red2 = np.array([170, 150, 150])
    upper_bright_red2 = np.array([180, 255, 255])

    # Define the range of white color in HSV
    lower_white = np.array([0, 0, 220])
    upper_white = np.array([180, 25, 255])

    # Create masks for the bright red and white LEDs
    mask_bright_red1 = cv2.inRange(hsv_image, lower_bright_red1, upper_bright_red1)
    mask_bright_red2 = cv2.inRange(hsv_image, lower_bright_red2, upper_bright_red2)
    mask_bright_red = cv2.bitwise_or(mask_bright_red1, mask_bright_red2)
    mask_white = cv2.inRange(hsv_image, lower_white, upper_white)
    mask_leds = cv2.bitwise_or(mask_bright_red, mask_white)

    # Create a mask to retain only the segmented part
    mask = mask_leds

    # Apply the mask to the original image to extract the segmented part
    segmented_image = cv2.bitwise_and(image, image, mask=mask)

    return segmented_image

def main():
    # Path to the input images
    image_path = '/content/drive/MyDrive/pre_preocessed/Image1_Bright_Red_and_White_LEDs_Enhanced.jpg'
    image_path1 = '/content/drive/MyDrive/pre_preocessed/Image2_Bright_Red_and_White_LEDs_Enhanced.jpg'

    # Perform image segmentation
    segmented_image = segment_image(image_path)
    segmented_image1 = segment_image(image_path1)

    # Display the segmented images
    cv2_imshow(segmented_image)
    cv2_imshow(segmented_image1)

    # Create a folder to save the segmented images
    output_folder = '/content/drive/MyDrive/output_nvidia1'
    os.makedirs(output_folder, exist_ok=True)

    # Save the segmented images
    cv2.imwrite(os.path.join(output_folder, 'segmented_image1.jpg'), segmented_image)
    cv2.imwrite(os.path.join(output_folder, 'segmented_image2.jpg'), segmented_image1)

if __name__ == "__main__":
    main()


In [ ]:
#EDGE MAPPING
from google.colab.patches import cv2_imshow
import cv2
import os

# Function to apply edge detection and save the result
def apply_edge_detection(image_path, output_folder):
    # Read the image
    original_image = cv2.imread(image_path)

    # Check if the image was loaded successfully
    if original_image is None:
        print(f"Error: Unable to read image at path: {image_path}")
        return

    # Convert the original image to grayscale
    gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

    # Perform Canny edge detection to detect edges
    edges = cv2.Canny(gray, 50, 150)

    # Retain the original color and intensity of the image after edge detection
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    result = cv2.addWeighted(original_image, 1, edges_rgb, 1, 0)

    # Save the result
    filename = os.path.basename(image_path)
    output_path = os.path.join(output_folder, f'edges_{filename}')
    cv2.imwrite(output_path, result)
    print(f"Edge-detected image saved to: {output_path}")

    # Display the original and edge-detected images
    cv2_imshow(original_image)
    cv2_imshow(result)

# Input image paths
image_paths = [
    '/content/drive/MyDrive/output_nvidia1/segmented_image1.jpg',
    '/content/drive/MyDrive/output_nvidia1/segmented_image2.jpg'
]

# Output folder
output_folder = '/content/drive/MyDrive/output_nvidia'

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Process each image
for image_path in image_paths:
    apply_edge_detection(image_path, output_folder)


In [ ]:
#ASYNC DETECTION
import cv2
import numpy as np
from matplotlib import pyplot as plt

def detect_blobs(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Set up the blob detector parameters
    params = cv2.SimpleBlobDetector_Params()

    # Change thresholds
    params.minThreshold = 10
    params.maxThreshold = 200

    # Filter by Area.
    params.filterByArea = True
    params.minArea = 150

    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0.1

    # Filter by Convexity
    params.filterByConvexity = True
    params.minConvexity = 0.87

    # Filter by Inertia
    params.filterByInertia = True
    params.minInertiaRatio = 0.01

    # Create a blob detector with the parameters
    detector = cv2.SimpleBlobDetector_create(params)

    # Detect blobs
    keypoints = detector.detect(gray_image)

    return keypoints

def compare_images(image1, image2):
    # Detect blobs in both images
    keypoints1 = detect_blobs(image1)
    keypoints2 = detect_blobs(image2)

    # Count the number of blobs in each image
    count1 = len(keypoints1)
    count2 = len(keypoints2)

    if count1 != count2:
        result_text = "Async: Different count of glowing LEDs"
    else:
        # Check if positions are the same
        same_positions = True
        for kp1, kp2 in zip(keypoints1, keypoints2):
            if abs(kp1.pt[0] - kp2.pt[0]) > 5 or abs(kp1.pt[1] - kp2.pt[1]) > 5:
                same_positions = False
                break

        if same_positions:
            result_text = "Sync: Same count and positions of glowing LEDs"
        else:
            result_text = "Async: Same count but different positions of glowing LEDs"

    # Draw detected blobs on the images
    im_with_keypoints1 = cv2.drawKeypoints(image1, keypoints1, np.array([]), (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    im_with_keypoints2 = cv2.drawKeypoints(image2, keypoints2, np.array([]), (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    # Display the images with detected blobs
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(im_with_keypoints1, cv2.COLOR_BGR2RGB))
    plt.title('Image 1: Detected Blobs')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(im_with_keypoints2, cv2.COLOR_BGR2RGB))
    plt.title('Image 2: Detected Blobs')
    plt.axis('off')

    plt.suptitle(result_text)
    plt.show()

# Load images
image1 = cv2.imread('/content/drive/MyDrive/output_nvidia/edges_segmented_image1.jpg')
image2 = cv2.imread('/content/drive/MyDrive/output_nvidia/edges_segmented_image2.jpg')

# Compare images
compare_images(image1, image2)
